## 错误处理

在程序运行的过程中，如果发生了错误，可以事先约定返回一个错误代码，这样，就可以知道是否有错，以及出错的原因。在操作系统提供的调用中，返回错误码非常常见。比如打开文件的函数open()，成功时返回文件描述符（就是一个整数），出错时返回-1  
用错误码来表示是否出错十分不便，因为函数本身应该返回的正常结果和错误码混在一起，造成调用者必须用大量的代码来判断是否出错

### try

高级语言通常都内置了一套try...except...finally...的错误处理机制，Python也不例外  
当我们认为某些代码可能会出错时，就可以用try来运行这段代码，如果执行出错，则后续代码不会继续执行，而是直接跳转至错误处理代码，即except语句块，执行完except后，如果有finally语句块，则执行finally语句块，至此，执行完毕

In [4]:
try:
    print('try...')
    r = 10 / 2
    print('result:', r)
except ZeroDivisionError as e:
    print('except:', e)
finally:
    print('finally...')
print('END')   

try...
result: 5.0
finally...
END


你还可以猜测，错误应该有很多种类，如果发生了不同类型的错误，应该由不同的except语句块处理。没错，可以有多个except来捕获不同类型的错误  
此外，如果没有错误发生，可以在except语句块后面加一个else，当没有错误发生时，会自动执行else语句

In [6]:
try:
    print('try...')
    r = 10 / int('2')
    print('result:', r)
except ValueError as e:
    print('valueError:', e)
except ZeroDivision as e:
    print('ZeroDivisionError:', e)
else:
    print('no error!')
finally:
    print('finally...')
print('END')

try...
result: 5.0
no error!
finally...
END


Python的错误其实也是class，所有的错误类型都继承自BaseException，所以在使用except时需要注意的是，它不但捕获该类型的错误，还把其子类也“一网打尽”  
第二个except永远也捕获不到UnicodeError，因为UnicodeError是ValueError的子类，如果有，也被第一个except给捕获了

In [8]:
try:
    print('try...')
    r = 10 / int('a')
    print('result:', r)
except ValueError as e:
    print('ValueError')
except UnicodeError as e:
    print('UnicodeError')

try...
ValueError


使用try...except捕获错误还有一个巨大的好处，就是可以跨越多层调用，比如函数main()调用foo()，foo()调用bar()，结果bar()出错了，这时，只要main()捕获到了，就可以处理

In [9]:
def foo(s):
    return 10 / int(s)
def bar(s):
    return foo(s) * 2
def main():
    try:
        bar('0')
    except Exception as e:
        print('Error:', e)
    finally:
        print('finally...')

### 调用栈

In [10]:
def foo(s):
    return 10 / in(s)
def bar(s):
    return foo(s) * 2
def main():
    bar('0')
    
main()

SyntaxError: invalid syntax (<ipython-input-10-b7b22b2667ba>, line 2)

### 记录错误

Python内置的logging模块可以非常容易地记录错误信息

In [1]:
import logging 
def foo(s):
    return 10 / int(s)
def bar(s):
    return foo(s) * 2
def main():
    try :
        bar('0')
    except Exception as e:
        logging.exception(e)

In [3]:
main()
print('END')

ERROR:root:division by zero
Traceback (most recent call last):
  File "<ipython-input-1-c38dd440fc1e>", line 8, in main
    bar('0')
  File "<ipython-input-1-c38dd440fc1e>", line 5, in bar
    return foo(s) * 2
  File "<ipython-input-1-c38dd440fc1e>", line 3, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero


END


### 抛出错误

如果要抛出错误，首先根据需要，可以定义一个错误的class，选择好继承关系，然后，用raise语句抛出一个错误的实例

In [4]:
class FooError(ValueError):
    pass
def foo(s):
    n = int(s)
    if n == 0:
        raise FooError('invalid value: %s' % s)
    return 10 / n

In [5]:
foo('0')

FooError: invalid value: 0

In [6]:
def foo(s):
    n = int(s)
    if n == 0:
        raise ValueError('invalid value: %s' % s)
    return 10 / n
def bar():
    try:
        foo('0')
    except ValueError as e:
        print('ValueError!')
        raise

In [7]:
bar()

ValueError!


ValueError: invalid value: 0

In [8]:
try:
    10 / 0
except ZeroDivisionError:
    raise ValueError('input error!')

ValueError: input error!

In [19]:
from functools import reduce
def str2num(s):
    return float(s)
def calc(exp):
    ss = exp.split('+')
    ns = map(str2num, ss)
    return reduce(lambda acc, x: acc + x, ns)
def main():
    r = calc('100 + 200 + 345')
    print('100 + 200 + 345 =', r)
    r = calc('99 + 88 + 7.6 ')
    print('99 + 88 + 7.6 =', r)

In [21]:
main()

100 + 200 + 345 = 645.0
99 + 88 + 7.6 = 194.6


## 调试

第一种方法简单直接粗暴有效，就是用print()把可能有问题的变量打印出来看看  

### 断言

凡是用print()来辅助查看的地方，都可以用断言（assert）来替代 

In [33]:
def foo(s):
    n = int(s)
    assert n != 0, 'n is zero!'
    return 10 / n
def main():
    foo('0')

In [34]:
main()

AssertionError: n is zero!

assert的意思是，表达式n != 0应该是True，否则，根据程序运行的逻辑，后面的代码肯定会出错。  
如果断言失败，assert语句本身就会抛出AssertionError

程序中如果到处充斥着assert，和print()相比也好不到哪去。不过，启动Python解释器时可以用-O参数来关闭assert

#### logging

把print()替换为logging是第3种方式，和assert比，logging不会抛出错误，而且可以输出到文件  
这就是logging的好处，它允许你指定记录信息的级别，有debug，info，warning，error等几个级别，当我们指定level=INFO时，logging.debug就不起作用了。同理，指定level=WARNING后，debug和info就不起作用了。这样一来，你可以放心地输出不同级别的信息，也不用删除，最后统一控制输出哪个级别的信息

In [37]:
import logging
logging.basicConfig(level=logging.INFO)
s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

ZeroDivisionError: division by zero

### pdb  
第4种方式是启动Python的调试器pdb，让程序以单步方式运行，可以随时查看运行状态  
类似gdb

### pdb.set_trace()

这个方法也是用pdb，但是不需要单步执行，我们只需要import pdb，然后，在可能出错的地方放一个pdb.set_trace()，就可以设置一个断点

### IDE

## 单元测试

单元测试是用来对一个模块、一个函数或者一个类来进行正确性检验的测试工作  
比如对函数abs()，我们可以编写出以下几个测试用例：  
输入正数，比如1、1.2、0.99，期待返回值与输入相同；  
输入负数，比如-1、-1.2、-0.99，期待返回值与输入相反； 
输入0，期待返回0；  
输入非数值类型，比如None、[]、{}，期待抛出TypeError。  
把上面的测试用例放到一个测试模块里，就是一个完整的单元测试。  
如果单元测试通过，说明我们测试的这个函数能够正常工作。如果单元测试不通过，要么函数有bug，要么测试条件输入不正确，总之，需要修复使单元测试能够通过

In [38]:
class Dict(dict):
    def __init__(self, **kw):
        super().__init__(**kw)
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)
    def __setattr__(self, key, value):
        self[key] = value

In [4]:
import unittest
from mydict import Dict
class TestDict(unittest.TestCase):
    def test_init(self):
        d = Dict(a=1, b='test')
        self.assertEqual(d.a, 1)
        self.assertEqual(d.b, 'test')
        self.assertTrue(isinstance(d, dict))
    def test_key(self):
        d = Dict()
        d.key = 'value'
        self.assertTrue('key' in d)
        self.assertEqual(d['key'], 'value')
    def test_keyerror(self):
        d = Dict()
        with self.assertRaisses(KeyError):
            value = d['empty']
    def test_attrerror(self):
        d = Dict()
        with self.assertRaise(AttributeError):
            value = d.empty
if __name__ == '__main__':
    unittest.main()

E
ERROR: C:\Users\Lenovo\AppData\Roaming\jupyter\runtime\kernel-e175019b-2063-48ea-9e79-9b3d4127a6ca (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\Lenovo\AppData\Roaming\jupyter\runtime\kernel-e175019b-2063-48ea-9e79-9b3d4127a6ca'

----------------------------------------------------------------------
Ran 1 test in 0.008s

FAILED (errors=1)


SystemExit: True

D:\Users\Lenovo\Anaconda2\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


另一种方法是在命令行通过参数-m unittest直接运行单元测试  
这是推荐的做法，因为这样可以一次批量运行很多单元测试，并且，有很多工具可以自动来运行这些单元测试。

### setUp与tearDown  
setUp()和tearDown()方法有什么用呢？设想你的测试需要启动一个数据库，这时，就可以在setUp()方法中连接数据库，在tearDown()方法中关闭数据库，这样，不必在每个测试方法中重复相同的代码

In [6]:
# _*_ coding: utf-8 _*_
import unittest
class Student(object):
    def __init__(self, name, score):
        self.name = name
        self.score = score
    def get_grade(self):
        if self.score >= 80:
            return 'A'
        if self.score >= 60:
            return 'B'
        return 'C'

In [9]:
class TestStudent(unittest.TestCase):
    def test_80_to_100(self):
        s1 = Student('Bart', 80)
        s2 = Student('Lisa', 100)
        self.assertEqual(s1.get_grade(), 'A')
        self.assertEqual(s2.get_grade(), 'A')
    def test_60_to_80(self):
        s1 = Student('Bart', 60)
        s2 = Student('Lisa', 79)
        self.assertEqual(s1.get_grade(), 'B')
        self.assertEqual(s2.get_grade(), 'B')
    def test_0_to_60(self):
        s1 = Student('Bart', 0)
        s2 = Student('Lisa', 59)
        self.assertEqual(s1.get_grade(), 'C')
        self.assertEqual(s2.get_grade(), 'C')
    def test_invalid(self):
        s1 = Student('Bart', -1)
        s2 = Student('Lisa', 101)
        with self.assertRaises(ValueError):
            s1.get_grade()
        with self.assertRaises(ValueError):
            s2.get_grade()
if __name__ == '__main__':
    unittest.main()

E
ERROR: C:\Users\Lenovo\AppData\Roaming\jupyter\runtime\kernel-d073d69b-d26f-4e22-ba6a-5f11c3b9ae36 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\Lenovo\AppData\Roaming\jupyter\runtime\kernel-d073d69b-d26f-4e22-ba6a-5f11c3b9ae36'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

D:\Users\Lenovo\Anaconda2\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 文档测试  

In [10]:
import re
m = re.search('(?<=abc)def', 'abcdef')
m.group(0)

'def'

既然这些代码本身就可以粘贴出来直接运行，那么，可不可以自动执行写在注释中的这些代码呢  
答案是肯定的

In [1]:
def abs(n):
    '''
    Function to get absolute value of number
    Example :
    >>> abs(1)
    1
    >>> abs(-1)
    1
    >> abs(0)
    0
    '''
    return n if n>= 0 else (-n)